In [26]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
import seaborn as sns
import numpy as np

In [23]:
random_state=42
cross_validation_folds = 5
n_jobs = 2
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [24]:
print('*'*40, "train")
print(train_df.info())
print('*'*40, 'test')
print(test_df.info())

**************************************** train
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1

In [39]:
dict(zip(train_df['RoofMatl'].cat.categories, range(len(combined['RoofMatl'].cat.categories))))
train_df['RoofMatl'].astype('category').cat.categories

Index(['ClyTile', 'CompShg', 'Membran', 'Metal', 'Roll', 'Tar&Grv', 'WdShake',
       'WdShngl'],
      dtype='object')

In [4]:
print('train', train_df.shape)
print('test', test_df.shape)

train (1460, 81)
test (1459, 80)


In [64]:
random_state=42
cross_validation_folds = 5
n_jobs = 2
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')
test_ids = test_df.Id
y = train_df.SalePrice
train_df.drop(['SalePrice'], inplace=True, axis=1)
combine = [train_df, test_df]
combined = pd.concat([train_df, test_df])
train_df.replace(['NaN', 'NA'], np.nan, inplace=True)
test_df.replace(['NaN', 'NA'], np.nan, inplace=True)
def fill_column_with_median_value(df, column):
    """
    fill numerical columns with median value
    fill categorial columns with most common values
    """
    if df[column].dtype=='float64':
        median_value = combined[column].median()
    elif df[column].dtype=='object':
        median_value = combined[column].mode()
        if pd.isnull(median_value):
            print('what the fuck')
    df[column] = df[column].fillna(median_value)
    print('is there still na value?', df[column].loc[df[column].isna()])
def find_columns_with_na_values(df):
    for column in df.columns:
        if df.loc[df[column].isnull()].size>0:
            yield column
for dataset in combine:
    dataset.drop(['Id', 'Alley', 'PoolQC', 'MiscFeature', 'Fence'], inplace=True, axis=1)
    for column in find_columns_with_na_values(dataset):
        print('column with null values ---', column, dataset[column].dtype)
        print(pd.isna(dataset.loc[dataset[column].isnull(), column].iloc[0]))
        fill_column_with_median_value(dataset, column)
X = pd.get_dummies(train_df)
X_test = pd.get_dummies(test_df)
missing_cols = set( X.columns ) - set( X_test.columns )
print("\tmissing columns", missing_cols)
print('drop missing columns')
# for column in missing_cols:
#     X.drop(column, inplace=True, axis=1)
print('X shape', X.shape)
print('X_test shape', X_test.shape)

column with null values --- LotFrontage float64
True
is there still na value? Series([], Name: LotFrontage, dtype: float64)
column with null values --- MasVnrType object
True


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [41]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 75 columns):
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non-null object
MasVnrType       1452 no

In [5]:
categorical_columns = list(filter(lambda col: train_df[col].dtype=='object', list(train_df.columns)))
mapping = dict()
# get the mapping between column labels to values
for column in categorical_columns:
    combined[column] = combined[column].astype('category')
    mapping[column] = dict(zip(combined[column].cat.categories, range(len(combined[column].cat.categories))))
print(mapping)
for dataset in combine:
    dataset.replace(mapping, inplace=True)
print(train_df.shape, test_df.shape)

{'MSZoning': {'C (all)': 0, 'FV': 1, 'RH': 2, 'RL': 3, 'RM': 4}, 'Street': {'Grvl': 0, 'Pave': 1}, 'LotShape': {'IR1': 0, 'IR2': 1, 'IR3': 2, 'Reg': 3}, 'LandContour': {'Bnk': 0, 'HLS': 1, 'Low': 2, 'Lvl': 3}, 'Utilities': {'AllPub': 0, 'NoSeWa': 1}, 'LotConfig': {'Corner': 0, 'CulDSac': 1, 'FR2': 2, 'FR3': 3, 'Inside': 4}, 'LandSlope': {'Gtl': 0, 'Mod': 1, 'Sev': 2}, 'Neighborhood': {'Blmngtn': 0, 'Blueste': 1, 'BrDale': 2, 'BrkSide': 3, 'ClearCr': 4, 'CollgCr': 5, 'Crawfor': 6, 'Edwards': 7, 'Gilbert': 8, 'IDOTRR': 9, 'MeadowV': 10, 'Mitchel': 11, 'NAmes': 12, 'NPkVill': 13, 'NWAmes': 14, 'NoRidge': 15, 'NridgHt': 16, 'OldTown': 17, 'SWISU': 18, 'Sawyer': 19, 'SawyerW': 20, 'Somerst': 21, 'StoneBr': 22, 'Timber': 23, 'Veenker': 24}, 'Condition1': {'Artery': 0, 'Feedr': 1, 'Norm': 2, 'PosA': 3, 'PosN': 4, 'RRAe': 5, 'RRAn': 6, 'RRNe': 7, 'RRNn': 8}, 'Condition2': {'Artery': 0, 'Feedr': 1, 'Norm': 2, 'PosA': 3, 'PosN': 4, 'RRAe': 5, 'RRAn': 6, 'RRNn': 7}, 'BldgType': {'1Fam': 0, '2fmCo

In [6]:
train_df.to_csv("train_df_processed.csv")

## Apply baseline model - Random Forest

In [11]:
? pd.fillna

Object `pd.fillna` not found.


In [36]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 75 columns):
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non-null object
MasVnrType       1452 no

In [7]:
rdf = GradientBoostingRegressor()
cv = cross_val_score(rdf, train_df, y, cv=cross_validation_folds, n_jobs=n_jobs, verbose=1)
print("baseline model with random forest, performance = {}".format(cv))
rdf.fit(X, y)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [46]:
y_pred = rdf.predict(X_test)
submission = pd.DataFrame({
    "Id": test_ids,
    "SalePrice": y_pred
})
submission.to_csv('house_price_result_baseline_random_forest.csv', index=False)

In [47]:
! kaggle competitions submit house-prices-advanced-regression-techniques -f house_price_result_baseline_random_forest.csv \
-m "baseline model with random forest, no feature engineering"

^C
User cancelled operation


In [17]:
sns.factorplot(data=y_pred)

NameError: name 'y_pred' is not defined